# Multiple Results


In this notebook we show how you can in a single request, have the LLM model return multiple results per prompt. This is useful for running experiments where you want to evaluate the robustness of your prompt and the parameters of your config against a particular large language model.


In [ ]:
!python -m pip install semantic-kernel==0.9.6b1

In [ ]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.OpenAI

In [ ]:
from semantic_kernel.contents import ChatHistory  # noqa: F401

if selectedService == Service.OpenAI or selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import (  # noqa: F401
        AzureChatCompletion,
        AzureChatPromptExecutionSettings,
        AzureTextCompletion,
        OpenAIChatCompletion,
        OpenAIChatPromptExecutionSettings,
        OpenAITextCompletion,
        OpenAITextPromptExecutionSettings,
    )
if selectedService == Service.HuggingFace:
    from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion  # noqa: F401

First, we will set up the text and chat services we will be submitting prompts to.


In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.utils.settings import azure_openai_settings_from_dot_env, openai_settings_from_dot_env

kernel = Kernel()

# Configure Azure LLM service
if selectedService == Service.AzureOpenAI:
    deployment, api_key, endpoint = azure_openai_settings_from_dot_env()
    azure_text_service = AzureTextCompletion(
        service_id="aoai_text", deployment_name="gpt-35-turbo-instruct", endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your text model (e.g. gpt-35-turbo-instruct or text-davinci-003)
    azure_chat_service = AzureChatCompletion(
        service_id="aoai_chat", deployment_name="gpt-35-turbo", endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model

# Configure OpenAI service
if selectedService == Service.OpenAI:
    api_key, org_id = openai_settings_from_dot_env()
    oai_text_service = OpenAITextCompletion(
        service_id="oai_text", ai_model_id="gpt-3.5-turbo-instruct", api_key=api_key, org_id=org_id
    )
    oai_chat_service = OpenAIChatCompletion(
        service_id="oai_chat", ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id
    )

# Configure Hugging Face service
if selectedService == Service.HuggingFace:
    hf_text_service = HuggingFaceTextCompletion(service_id="hf_text", ai_model_id="distilgpt2", task="text-generation")

Next, we'll set up the completion request settings for text completion services.


In [ ]:
oai_text_prompt_execution_settings = OpenAITextPromptExecutionSettings(
    service="oai_text",
    extension_data={
        "max_tokens": 80,
        "temperature": 0.7,
        "top_p": 1,
        "frequency_penalty": 0.5,
        "presence_penalty": 0.5,
        "number_of_responses": 3,
    },
)

## Multiple Open AI Text Completions


In [ ]:
if selectedService == Service.OpenAI:
    chat = ChatHistory()
    chat.add_user_message("What is the purpose of a rubber duck?")
    #results = await oai_text_service.complete(chat_history=chat, settings=oai_text_prompt_execution_settings)
    results = await oai_text_service.complete(chat, settings=oai_text_prompt_execution_settings)
    i = 1
    for result in results:
        print(f"Result {i}: {result}")
        i += 1

## Multiple Azure Open AI Text Completions


In [8]:
if selectedService == Service.AzureOpenAI:
    chat = ChatHistory()
    chat.add_user_message("provide me a list of possible meanings for the acronym 'ORLD'")
    results = await azure_text_service.complete(chat_history=chat, settings=oai_text_prompt_execution_settings)
    i = 1
    for result in results:
        print(f"Result {i}: {result}")
        i += 1

## Multiple Hugging Face Text Completions


In [9]:
if selectedService == Service.HuggingFace:
    from semantic_kernel.connectors.ai.hugging_face.hf_prompt_execution_settings import (
        HuggingFacePromptExecutionSettings,
    )

    hf_prompt_execution_settings = HuggingFacePromptExecutionSettings(
        service_id="hf_text", extension_data={"max_new_tokens": 80, "temperature": 0.7, "top_p": 1}
    )

In [10]:
if selectedService == Service.HuggingFace:
    prompt = "The purpose of a rubber duck is"
    chat = ChatHistory()
    chat.add_user_message(prompt)
    results = await hf_text_service.complete(chat_history=chat, prompt_execution_settings=hf_prompt_execution_settings)
    print("".join(results))

Here, we're setting up the settings for Chat completions.


In [11]:
oai_chat_prompt_execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="oai_chat",
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3,
)

## Multiple OpenAI Chat Completions


In [12]:
if selectedService == Service.OpenAI:
    chat = ChatHistory()
    chat.add_user_message(
        "It's a beautiful day outside, birds are singing, flowers are blooming. On days like these, kids like you..."
    )
    results = await oai_chat_service.complete_chat(chat_history=chat, settings=oai_chat_prompt_execution_settings)
    i = 0
    for result in results:
        print(f"Result {i+1}: {str(result)}")
        i += 1

Result 1: should go outside and enjoy the fresh air! Play some games, ride your bike, or have a picnic with your friends and family. Take advantage of the beautiful weather and make some fun memories. Don't forget to appreciate the beauty of nature all around you.
Result 2: should go outside and play! Enjoy the fresh air, soak up the sunshine, and have some fun. Whether it's riding bikes, playing tag, or having a picnic in the park, make the most of this beautiful day and create some wonderful memories. Don't forget to take a moment to appreciate the beauty of nature around you and be grateful for all the little things that bring joy to your life
Result 3: should go outside and enjoy the fresh air! Play in the sunshine, run around and get some exercise. Take a walk in nature, ride your bike, or have a picnic in the park. The possibilities are endless on a beautiful day like today. Don't waste it sitting inside staring at screens - go out and make some memories!


## Multiple Azure OpenAI Chat Completions


In [13]:
az_oai_prompt_execution_settings = AzureChatPromptExecutionSettings(
    service_id="aoai_chat",
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3,
)

In [14]:
if selectedService == Service.AzureOpenAI:
    content = (
        "Tomorrow is going to be a great day, I can feel it. I'm going to wake up early, go for a run, and then..."
    )
    chat = ChatHistory()
    chat.add_user_message(content)
    results = await azure_chat_service.complete_chat(chat_history=chat, settings=az_oai_prompt_execution_settings)
    i = 0
    for result in results:
        print(f"Result {i+1}: {str(result)}")
        i += 1

## Streaming Multiple Results

Here is an example pattern if you want to stream your multiple results. Note that this is not supported for Hugging Face text completions at this time.


In [21]:
if selectedService == Service.OpenAI:
    import os
    import time

    from IPython.display import clear_output

    # Determine the clear command based on OS
    clear_command = "cls" if os.name == "nt" else "clear"

    chat = ChatHistory()
    chat.add_user_message("what is the purpose of a rubber duck?")
    chat_prompt = "what is the purpose of a rubber duck?"
    stream = oai_text_service.complete_stream(prompt=chat_prompt, settings=oai_text_prompt_execution_settings)
    number_of_responses = oai_text_prompt_execution_settings.number_of_responses
    texts = [""] * number_of_responses

    last_clear_time = time.time()
    clear_interval = 0.5  # seconds

    # Note: there are some quirks with displaying the output, which sometimes flashes and disappears.
    # This could be influenced by a few factors specific to Jupyter notebooks and asynchronous processing.
    # The following code attempts to buffer the results to avoid the output flashing on/off the screen.

    async for results in stream:
        current_time = time.time()

        # Update texts with new results
        for idx, result in enumerate(results):
            if idx < number_of_responses:
                texts[idx] += str(result)

        # Clear and display output at intervals
        if current_time - last_clear_time > clear_interval:
            clear_output(wait=True)
            for idx, text in enumerate(texts):
                print(f"Result {idx + 1}: {text}")
            last_clear_time = current_time

    print("----------------------------------------")

Result 1: 




TheTheA purpose purpose rubber of of duck a a is rubber rubber often duck duck used is is as primarily primarily a as for bath a use toy bath as for toy a children for bath. children toy It,. can but It also is be typically used made it can also be of soft used as, as a stress relie a flexiblever stress or-rel rubber and deskiever designed decoration or to for desktop resemble adults decoration a. for duck In adults
Result 2: 
Result 3: 
----------------------------------------
